# Real Time Detections 

1. Setup 
2. Load the model
3. Real Time Detections

## Setup & Load Model

<div class="alert alert-block alert-warning">
<b>Attention:</b> Set model name
</div>

In [ ]:
# Import dependencies
import os, random
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np
from matplotlib import pyplot as plt
import time
%matplotlib inline


# Model Name and labelmap
CUSTOM_MODEL_NAME = 'centernet_resnet50_v5'
LABEL_MAP_NAME = 'label_map.pbtxt'

# Paths
paths = {
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'), 
    
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-7')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

## Real Time Detections

### Detection Tensorflow

In [ ]:

cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)    
    imH, imW, _ = frame.shape
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=4,
                min_score_thresh=.7,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

### Detection Counter

In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Text settings
color = (10, 255, 0)
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale_big = 1
font_scale_small = 0.5
thickness = 1
lineType = cv2.LINE_AA

# Text position
position_fps = (10,50)


# Initialize frame rate calculation & total count
frame_rate_calc = 1
freq = cv2.getTickFrequency()
total_count = 0

# min threshold
MIN_CONF_THRESH = 0.9

while cap.isOpened(): 
    
    # Set the current count to 0
    current_count=0
    
    # Start timer (for calculating frame rate)
    t1 = cv2.getTickCount()
    
    ret, frame = cap.read()
    image_np = np.array(frame)    
    imH, imW, _ = frame.shape
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections


    # SET MIN SCORE THRESH TO MINIMUM THRESHOLD FOR DETECTIONS
    
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    scores = detections['detection_scores']
    boxes = detections['detection_boxes']
    classes = detections['detection_classes']
    count = 0
    number_cola = 0
    number_pepsi = 0
    
    print(boxes.size)
    
    
    for i in range(len(scores)):
        if ((scores[i] > MIN_CONF_THRESH) and (scores[i] <= 1.0)):
           
                        
            ymin = int(max(1,(boxes[i][0] * imH)))
            xmin = int(max(1,(boxes[i][1] * imW)))
            ymax = int(min(imH,(boxes[i][2] * imH)))
            xmax = int(min(imW,(boxes[i][3] * imW)))    
            
            # Draw boders
            cv2.rectangle(frame, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)
            
            # Draw label
            object_name = category_index[int(classes[i])+1]['name'] 
            label = '%s: %d%%' % (object_name, int(scores[i]*100)) 
            labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1, 2) 
            label_ymin = max(ymin, labelSize[1] + 10) 
            cv2.rectangle(frame, (xmin, label_ymin-labelSize[1]), (xmin+labelSize[0]-50, label_ymin+baseLine), color, cv2.FILLED) 
            cv2.putText(frame, label, (xmin, label_ymin-5), font, font_scale_small,(0,0,0), thickness, lineType) 
            
            # Counter
            if (object_name == 'Cola'):
                number_cola += 1;
            elif (object_name == 'Pepsi'):
                number_pepsi += 1;
            count += 1

    # Left Corner
    cv2.rectangle(frame, (10, 10), (180, 120), (255, 255, 255), cv2.FILLED)
    cv2.putText (frame,'Objects Detected : ' + str(count),(10,25),font, font_scale_small, (0,0,0), thickness, lineType)    
    cv2.putText(frame,'FPS: {0:.2f}'.format(frame_rate_calc), (10,50), font, font_scale_small,(0,0,0), thickness, lineType)
    cv2.putText(frame,'Number of Cola: ' + str(number_cola), (10, 80), font, font_scale_small,(0,0,0), thickness, lineType)
    cv2.putText(frame,'Number of Pepsi: '+ str(number_pepsi), (10, 100), font, font_scale_small,(0,0,0), thickness, lineType)
    cv2.imshow('Objects Detector', frame)    
 
    # Calculate framerate    
    t2 = cv2.getTickCount()
    time1 = (t2-t1)/freq
    frame_rate_calc= 1/time1    
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

### Detection Images

In [ ]:
def detection_single_img(img_path):
    img = cv2.imread(img_path)
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+1,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=3,
                min_score_thresh=.8,
                line_thickness=20)

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
detection_single_img(IMAGE_PATH)

In [ ]:
def detection_9_img(img_list):
    
    plt.figure(figsize=(20, 20))
    
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)        
    
        img = cv2.imread(img_list[i])
        image_np = np.array(img)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+1,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=3,
                    min_score_thresh=.8,
                    line_thickness=20)

        plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        

def choose_random_imgs(number_imgs, path):  
    imgs_list = []
    f = 1
    while f <= number_imgs:
        file = random.choice(os.listdir(path))  
        if file.split('.')[-1] == 'jpg' and file not in imgs_list:
            f += 1
            imgs_list.append(os.path.join(paths['IMAGE_PATH'], 'test', file))
    return imgs_list

In [ ]:
img_test_set = choose_random_imgs(9, os.path.join(paths['IMAGE_PATH'], 'test'))
detection_9_img(img_test_set)